In [3]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import re

### Get gene ID mapping directly from genbank
gb = SeqIO.read('/home/laurence/ME/data/references/NC_000913.3.gb','gb')

patt_gi = re.compile(r'GI:(\d+)')
patt_uniprot = re.compile(r'UniProtKB/Swiss-Prot:([A-Z]\S+)')
element_types = {'CDS','tRNA','rRNA','ncRNA','tmRNA'}
full_seq = str(gb.seq)
nSeq = len(gb.seq)

k_gene = 0
rows = []
for f in [f for f in gb.features if f.type in element_types]:
    locus_tags = f.qualifiers['locus_tag']
    locus_tags = [l.strip() for l in locus_tags]
    uniprots = [patt_uniprot.findall(x) for x in f.qualifiers['db_xref']]
    uniprots = [uni[0] for uni in uniprots if uni]
    gis = [patt_gi.findall(x) for x in f.qualifiers['db_xref']]
    gis = [gi[0] for gi in gis if gi]
    genes = f.qualifiers['gene']
    genes += f.qualifiers['gene_synonym'][0].split(';')
    genes = [g.strip() for g in genes]
        
    if len(gis)==0:
        gis = [np.nan]
    if len(genes)==0:
        genes = [np.nan]
    if len(locus_tags)==0:
        locus_tags = [np.nan]
    for uniprot in uniprots:
        for gi in gis:
            for gene in genes:
                for locus in locus_tags:
                    row = {'locus_tag':locus,
                           'GI':gi,
                           'gene':gene,
                           'uniprot':uniprot}
                    rows.append(row)

df_annot = pd.DataFrame(rows)
df_annot = df_annot.drop_duplicates()
print df_annot.shape
print df_annot[ df_annot.GI.isnull()].shape
print df_annot[ df_annot.gene.isnull()].shape
print df_annot[ df_annot.locus_tag.isnull()].shape

(16017, 4)
(532, 4)
(0, 4)
(0, 4)


In [6]:
df_annot.head(2)

,GI,gene,locus_tag,uniprot
0,16127995,thrL,b0001,P0AD86
1,16127995,ECK0001,b0001,P0AD86


In [5]:
df_annot.to_csv('/home/laurence/ME/data/dynamicME/gene_mapping.csv', index=False)